<a href="https://colab.research.google.com/github/AndromedaOMA/Advanced_Analytics_with_Apache_Spark---E.On_Software_Development/blob/main/Laboratory_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark oferă:
    
1.  Un obiect de tip **Spark Context**, folosit pentru interacțiunea cu Spark.
2.  Un obiect de bază pentru codul SQL și citirea datelor, numit **Spark Session**.
3.  Un obiect de lucru cu date, numit **Data Frame**, având o interfață similară cu cel de *Pandas*.
4.  Funcții gata implementate pentru transformări și expresii.
5.   Abilitatea de a construi propriile funcții de transformare în *Python*.



---

# **Pregătire mediu de lucru**

Stabilirea conexiunii dintre Google drive si Colab notebook și importarea/instalarea pachetelor și librăriilor necesare mai departe.

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://dlcdn.apache.org/spark/
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"
import findspark
findspark.init()
findspark.find()
import pyspark

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
47 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [25]:
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
spark= SparkSession \
.builder \
.getOrCreate()
spark

---

# **Data Frame**

**Spark SQL** lucrează datele sub forma unui **tabel**.

Librăria **PySpark** oferă un clasă care reprezintă un tabel de date, unde fiecare coloană conține același tip de date, numită **Data Frame**, similar cu ce oferă librăria **Pandas**.

Fiind concepute pentru Big Data, un Data Frame nu încarcă datele.
El reține doar locația și tipul lor.
La fiecare  transformare de date, noul Data Frame va fi creat care reține locația și tipul datelor, precum și lanțul de transformări.
Transformările vor fi efectuate **doar** în momentul scrierii sau afișării datelor de către executori.

 ➢ Această schema se poate defini de dezolator după un anume format

 ➢ Pentru a dezactiva determinarea automata a tipurilor de date, la citire, se poate furniza schema

 ➢ Principalele tipuri de date din Spark (Lista completă: https://spark.apache.org/docs/latest/sql-ref-datatypes.html ):

1. integer
2. string
3. float
4. double
5. boolean
6. date
7. timestamp
8. array<tip element>
9. map<tip cheie, tip valoare>
10. struct<coloana1:tip 1, coloana2:tip 2,...>

## Crearea unui Data Frame Spark dintr-o listă de Python.

In [27]:
data = [
    ['Vali', 23, 'Programator', 4, None, ['Sport', 'Boardgames']],
    ['Vlad', 34, 'Instalator', 11, None, ['Alergare']],
    ['Bea', 29, 'Reporter', 7, True, None]
]

data_df_python = spark.createDataFrame(data)

data_df_python.show()

+----+---+-----------+---+----+-------------------+
|  _1| _2|         _3| _4|  _5|                 _6|
+----+---+-----------+---+----+-------------------+
|Vali| 23|Programator|  4|NULL|[Sport, Boardgames]|
|Vlad| 34| Instalator| 11|NULL|         [Alergare]|
| Bea| 29|   Reporter|  7|true|               NULL|
+----+---+-----------+---+----+-------------------+



## Formatele JSON, Parquet și CSV

În Big Data, în general, datele sunt stocate extern în unul sau mai multe formate. PySpark oferă metode
pentru citirea celor mai populare și des utilizare formate, precum JSON, Parquet sau CSV.

### Citirea fișierelor de tip JSON Lines.

In [28]:
path_json = "/content/drive/MyDrive/FP-EON/practice/IN/json"
data_df_json = spark.read.format('json').load(path_json)
data_df_json.show()

+--------------------+-------+-----------+--------------------+------+-------+
|               extra|inactiv|       nume|            ocupatie|varsta|vechime|
+--------------------+-------+-----------+--------------------+------+-------+
|            [PV, EV]|   NULL|     Andrei|Specialist marketing|    38|     13|
|   [3D Printer, WII]|   NULL|  Alexandru|      Specialist HR |    34|      8|
| [AC, EV, 5G Router]|   NULL|     Adrian|       Inginer civil|    45|     23|
|              [XBOX]|   NULL|      Alin |   Vânzător  retail |    26|      2|
|[5G Router, 3D Pr...|   NULL|      Anton|     Manager proiect|    40|     15|
|            [EV, AC]|   NULL|        Ana|  Muncitor alimentar|    35|      9|
|[PC, 3D Printer, AC]|  false|     Bogdan|           Farmacist|    50|     32|
|                NULL|   true|  Cătălin  |        Medic primar|    20|      0|
|      [AC, XBOX, PV]|   NULL|     Cosmin|           Farmacist|    49|     31|
|[XBOX, 3D Printer...|  false|   Cristian|    Asiste

### Citirea fișierelor de tip Parquet.

In [29]:
path_parquet = "/content/drive/MyDrive/FP-EON/practice/IN/parquet"
data_df_parquet = spark.read.format('parquet').load(path_parquet)
data_df_parquet.show()

+------------+------+--------------------+-------+-------+--------------------+
|        nume|varsta|            ocupatie|vechime|inactiv|               extra|
+------------+------+--------------------+-------+-------+--------------------+
|       Daria|    33|   Inginer  mecanic |      7|   NULL|                [EV]|
|     Delia  |    20|        Medic primar|      0|   true|                NULL|
|       Diana|    45|   Asistent  social |     23|   NULL|[XBOX, 3D Printer...|
|       Doina|    37|Specialist  marke...|     11|   NULL|          [WII, PS5]|
|      Elena |    25|            Contabil|      1|   NULL|                NULL|
|     Eliza  |    23|          Instalator|      0|   NULL|                NULL|
|     Emilia |    25|            Contabil|      1|   NULL|                NULL|
|   Eugenia  |    24|        Programator |      1|   NULL|                NULL|
|     Felicia|    34|      Specialist  HR|      8|   NULL|          [WII, PS5]|
|    Florina |    28|               Șofe

### Citirea fișierelor de tip CSV cu rând de Header.

Putem schimba și setările implicite de citire. De exemplu, implicit, Spark ignoră Header-ul din fișiere CSV.

In [ ]:
path_csv = 'path/to/folder/or/file'
data_df_csv = spark.read.format('csv').option('header','true').load(path_csv)



---


# **Schema datelor**

Întrucât Spark SQL a fost conceput pentru date structurate, el încearcă să determine tipurile de coloane, numită schema datelor, în mod automat, din sursele de date la citire. Acest proces nu este necesar pentru formate structurate ca Parquet, dar necesar pentru obiecte Python sau date semi-structurate, JSON și CSV.

## Definirea unei scheme a datelor

PySpark oferă un și un mod programatic de a defini o schemă a datelor:

In [30]:
from pyspark.sql import types as types

data_schema = T.StructType([
    T.StructField('nume', T.StringType(), False),
    T.StructField('varsta', T.IntegerType(), False),
    T.StructField('ocupatie', T.StringType(), False),
    T.StructField('vechime',T.IntegerType(),True),
    T.StructField('inactiv', T.BooleanType(), True),
    T.StructField('extra', T.ArrayType(T.StringType()), True)
])

Dacă se furnizează la citire schema, Spark va ignora sau pune NULL pe coloanele care nu se potrivesc sau nu există:

In [31]:
data_df = spark.createDataFrame(data, schema=data_schema)
# or
# data_df = spark.read.format('json').schema(data_schema).load('/path/to/folder/or/file')

## Data Frame – Schema Datelor

Fie prin detectare automată, fie prin furnizarea la citire, fiecare Data Frame va avea întotdeauna o structură a
datelor foarte bine definită. Pentru fiecare coloană avem numele, tipul de dată și dacă se permit valori de ”NULL”.

➢ Pentru a afișa structură a datelor / schema datelor pentru un Data Frame, folosim:

In [33]:
data_df.printSchema()

root
 |-- nume: string (nullable = false)
 |-- varsta: integer (nullable = false)
 |-- ocupatie: string (nullable = false)
 |-- vechime: integer (nullable = true)
 |-- inactiv: boolean (nullable = true)
 |-- extra: array (nullable = true)
 |    |-- element: string (containsNull = true)



OBS: Dacă avem de lucrat cu date care nu au o schemă bine definită, fie date nestructurate, fie date semi-
structurate cu multe abateri, atunci NU folosim Data Frame-ul din Spark SQL.

## Data Frame – Afișarea Datelor

De multe ori ne găsim în situația în care este nevoie să depanăm procesul de transformare a datelor. Așadar, Spark
ofertă metode pentru acest scop.


➢ Pentru a printa datele la consolă, folosim:

In [36]:
data_df.show()

+----+------+-----------+-------+-------+-------------------+
|nume|varsta|   ocupatie|vechime|inactiv|              extra|
+----+------+-----------+-------+-------+-------------------+
|Vali|    23|Programator|      4|   NULL|[Sport, Boardgames]|
|Vlad|    34| Instalator|     11|   NULL|         [Alergare]|
| Bea|    29|   Reporter|      7|   true|               NULL|
+----+------+-----------+-------+-------+-------------------+



In [35]:
display(data_df.toPandas())

,nume,varsta,ocupatie,vechime,inactiv,extra
0,Vali,23,Programator,4,None,"[Sport, Boardgames]"
1,Vlad,34,Instalator,11,None,[Alergare]
2,Bea,29,Reporter,7,True,None


**Atenție!**

Datele sunt transferate mai întâi de la executori pe Spark Driver înainte de afișare, așadar această
metodă limitează numărul de date afișate pentru a evita prăbușirea mașinii principale.

## Data Frame – Colectarea Datelor

Alte metode pe care Spark le ofertă, nu numai pentru depănarea datelor, dar și pentru rarele situații când am redus datele și vrem să prelucrăm cu alte librării, sunt cele de colectare a datelor în obiecte de Python pe mașina Master.

➢ Pentru a colecta datele într-o listă de Python, folosim:

In [38]:
data_list = data_df.collect()
display(data_list)

[Row(nume='Vali', varsta=23, ocupatie='Programator', vechime=4, inactiv=None, extra=['Sport', 'Boardgames']),
 Row(nume='Vlad', varsta=34, ocupatie='Instalator', vechime=11, inactiv=None, extra=['Alergare']),
 Row(nume='Bea', varsta=29, ocupatie='Reporter', vechime=7, inactiv=True, extra=None)]

➢ Pentru a colecta datele într-un obiect de Data Frame, dar a librăriei Pandas, folosim:

In [41]:
data_pandas_pdf = data_df.toPandas()
display(data_pandas_pdf)

,nume,varsta,ocupatie,vechime,inactiv,extra
0,Vali,23,Programator,4,None,"[Sport, Boardgames]"
1,Vlad,34,Instalator,11,None,[Alergare]
2,Bea,29,Reporter,7,True,None


**Atenție!**

Datele sunt transferate mai întâi de la executori pe Spark Driver la colectare. Dacă sunt prea multe
date, există pericolul ca procesul de Spark Driver să nu mai facă față și să fie terminat de către sistem.

## Scrierea Datelor

În Big Data, după procesare, datele sunt stocate extern în unul sau mai multe formate. PySpark oferă metode pentru scrierea celor mai populare și des utilizare formate.

➢ Scrierea fișierelor de tip Parquet.

In [ ]:
data_df.write.format('parquet').save('/path/to/save/folder')

➢ Scrierea fișierelor de tip JSON Lines.


In [ ]:
data_df.write.format('json').save('/path/to/save/folder/')

Putem schimba și setările implicite de citire. De exemplu, implicit, Spark nu scrie Header-ul în fișiere CSV.

➢ Scrierea fișierelor de tip CSV cu rând de Header.

In [ ]:
data_df.write.format('csv').option('header', 'true').save('/path/to/save/folder')

OBS: Dacă avem mai multe setări, adăugăm la lanț câte un apel la funcția option pentru fiecare setare. O listă cu toate opțiunile disponibile pentru fiecare format găsiți la https://spark.apache.org/docs/latest/sql-data-sources.html.